# Spain Electricity Shortfall Challenge - Kaggle submission



## Regression Project Student Solution

© Explore Data Science Academy

---

The works in this notebook were done for the purpose of making a Kaggle submission which was part of the requirements for the successful completion of the project.

In [1]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np   
import pandas as pd   

# Libraries for data preparation and model building
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import *
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


from tqdm import tqdm
import math
import random

#Visualization Packages
from scipy import stats
from scipy.stats import norm, skew 
import seaborn as sns 
from matplotlib import pyplot as plt


# Import pickle used to save trained model
import pickle

## Data Cleaning and Feature Engineering on Test Data

In [2]:
# Load the test data
df_test = pd.read_csv('Data/df_test.csv')
df_test.tail()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min
2915,11678,2018-12-31 09:00:00,0.333333,level_9,0.0,2.000000,81.666667,49.666667,80.000000,1.333333,...,279.816667,281.483333,278.140000,270.816667,273.210000,276.483333,276.150000,279.816667,274.910000,271.150000
2916,11679,2018-12-31 12:00:00,0.333333,level_8,0.0,1.333333,61.000000,28.333333,56.666667,1.000000,...,286.483333,287.816667,286.150000,278.150000,278.443333,285.816667,278.816667,287.150000,283.156667,280.483333
2917,11680,2018-12-31 15:00:00,1.000000,level_6,0.0,3.000000,47.000000,26.333333,0.000000,0.666667,...,289.483333,288.816667,288.820000,284.150000,285.073333,288.150000,285.816667,290.816667,287.733333,286.483333
2918,11681,2018-12-31 18:00:00,1.000000,level_6,0.0,2.000000,52.666667,56.666667,0.000000,0.666667,...,285.816667,285.150000,284.473333,280.150000,281.626667,283.150000,282.816667,287.483333,283.813333,282.150000
2919,11682,2018-12-31 21:00:00,1.333333,level_10,0.0,2.333333,61.666667,69.333333,0.000000,1.333333,...,283.816667,276.816667,281.133333,276.150000,276.450000,278.483333,276.816667,283.816667,276.623333,276.483333


In [3]:
# Dropping the unnamed feature
df_test = df_test.drop(['Unnamed: 0'], axis = 1)

In [4]:
#Eliminating Null values

df_test['Valencia_pressure'] = df_test['Valencia_pressure'].fillna(df_test['Valencia_pressure'].mean())

In [5]:
# Confirming that the null values have been replaced

df_test.isnull().sum()

time                    0
Madrid_wind_speed       0
Valencia_wind_deg       0
Bilbao_rain_1h          0
Valencia_wind_speed     0
Seville_humidity        0
Madrid_humidity         0
Bilbao_clouds_all       0
Bilbao_wind_speed       0
Seville_clouds_all      0
Bilbao_wind_deg         0
Barcelona_wind_speed    0
Barcelona_wind_deg      0
Madrid_clouds_all       0
Seville_wind_speed      0
Barcelona_rain_1h       0
Seville_pressure        0
Seville_rain_1h         0
Bilbao_snow_3h          0
Barcelona_pressure      0
Seville_rain_3h         0
Madrid_rain_1h          0
Barcelona_rain_3h       0
Valencia_snow_3h        0
Madrid_weather_id       0
Barcelona_weather_id    0
Bilbao_pressure         0
Seville_weather_id      0
Valencia_pressure       0
Seville_temp_max        0
Madrid_pressure         0
Valencia_temp_max       0
Valencia_temp           0
Bilbao_weather_id       0
Seville_temp            0
Valencia_humidity       0
Valencia_temp_min       0
Barcelona_temp_max      0
Madrid_temp_

<br>

### Modifying the time feature

The `time` feature is mapped into a datetime format which is the appropriate data type for date and time features and other features can be created from the date time feature that can be used in the model training.

In [6]:
df_test['time'] = pd.to_datetime(df_test['time'])
df_test.time

0      2018-01-01 00:00:00
1      2018-01-01 03:00:00
2      2018-01-01 06:00:00
3      2018-01-01 09:00:00
4      2018-01-01 12:00:00
               ...        
2915   2018-12-31 09:00:00
2916   2018-12-31 12:00:00
2917   2018-12-31 15:00:00
2918   2018-12-31 18:00:00
2919   2018-12-31 21:00:00
Name: time, Length: 2920, dtype: datetime64[ns]

### Extracting other Datetime feature from the time feature

In [7]:
column_list =[]

# second
df_test['second'] = df_test['time'].dt.second

# minute
df_test['minute'] = df_test['time'].dt.minute

# hour
df_test['hour'] = df_test['time'].dt.hour

# day
df_test['Day'] = df_test['time'].dt.day

# month
df_test['Month'] = df_test['time'].dt.month

# year
df_test['Year'] = df_test['time'].dt.year


# adding the new features to the dataset 
column_list = ['time', 'second', 'minute', 'hour','Day','Month','Year'] + list(df_test.columns[1:-6])



df_test = df_test[column_list]


<br>

**Dropping the `second` and `minute` columns from the dataframe**

In [8]:
df_test.drop(columns=['second', 'minute'], inplace=True)

The `Valencia_wind_deg` seems to be a categorical data represented by different levels. This levels, which are represented by integers, can be extracted for the purpose of encoding to train the machine learning model.

In [9]:
# Extracting the digit from the strings of levels
df_test['Valencia_wind_deg'] = df_test['Valencia_wind_deg'].str.extract('(\d+)')

# Converting the extracted digits to integers
df_test['Valencia_wind_deg'] = pd.to_numeric(df_test['Valencia_wind_deg'])

# Checking the output
df_test['Valencia_wind_deg']

0        8
1        8
2        7
3        7
4        7
        ..
2915     9
2916     8
2917     6
2918     6
2919    10
Name: Valencia_wind_deg, Length: 2920, dtype: int64

<br>

`Seville_pressure` is another feature in the dataset that is still of type object, which is a string. The same process as above will be repeated on this feature to convert it to numeric.

In [10]:
# Extracting the digit from the strings of levels
df_test['Seville_pressure'] = df_test['Seville_pressure'].str.extract('(\d+)')

# Converting the extracted digits to integers
df_test['Seville_pressure'] = pd.to_numeric(df_test['Seville_pressure'])

# Checking the output
df_test['Seville_pressure']

0       25
1       25
2       25
3       25
4       25
        ..
2915    24
2916    24
2917    23
2918    23
2919    23
Name: Seville_pressure, Length: 2920, dtype: int64

### Selected Features

The selected features are those obtained in the cell above which were the features gotten after the elimination of highly correlated features using Variance Inflation Factor (VIF) and after testing them on the regression model, a better performance was obtained byt the random forest model.

These features will be listed out as list and will be useful in identifying features to use in subsequent use of the model.

In [11]:
final_features = ['hour', 'Day', 'Month', 'Year', 'Madrid_wind_speed', 
                  'Valencia_wind_deg', 'Bilbao_rain_1h', 'Valencia_wind_speed', 
                  'Seville_humidity', 'Madrid_humidity', 'Bilbao_clouds_all', 
                  'Bilbao_wind_speed', 'Seville_clouds_all', 'Bilbao_wind_deg', 
                  'Barcelona_wind_speed', 'Barcelona_wind_deg', 'Madrid_clouds_all', 
                  'Seville_wind_speed', 'Barcelona_rain_1h', 'Seville_pressure', 
                  'Seville_rain_1h', 'Bilbao_snow_3h', 'Barcelona_pressure', 
                  'Seville_rain_3h', 'Madrid_rain_1h', 'Barcelona_rain_3h', 
                  'Valencia_snow_3h', 'Madrid_weather_id', 'Barcelona_weather_id', 
                  'Bilbao_pressure', 'Seville_weather_id', 'Valencia_pressure', 
                  'Seville_temp_max', 'Madrid_pressure', 'Bilbao_weather_id', 
                  'Valencia_humidity', 'Barcelona_temp_min', 'Bilbao_temp_max']

### Final test data

In [12]:
test_data = df_test[final_features]
test_data

,hour,Day,Month,Year,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,...,Barcelona_weather_id,Bilbao_pressure,Seville_weather_id,Valencia_pressure,Seville_temp_max,Madrid_pressure,Bilbao_weather_id,Valencia_humidity,Barcelona_temp_min,Bilbao_temp_max
0,0,1,1,2018,5.000000,8,0.0,5.000000,87.000000,71.333333,...,800.000000,1025.666667,800.000000,1013.148351,284.483333,1030.000000,801.000000,46.333333,286.816667,285.150000
1,3,1,1,2018,4.666667,8,0.0,5.333333,89.000000,78.000000,...,800.333333,1026.666667,800.000000,1013.148351,282.483333,1030.333333,721.000000,53.666667,283.483333,284.150000
2,6,1,1,2018,2.333333,7,0.0,5.000000,89.000000,89.666667,...,800.000000,1025.333333,800.333333,1013.148351,280.816667,1030.666667,800.000000,42.000000,281.816667,282.150000
3,9,1,1,2018,2.666667,7,0.0,5.333333,93.333333,82.666667,...,800.000000,1025.000000,800.333333,1013.148351,281.150000,1032.333333,801.333333,45.000000,282.150000,284.483333
4,12,1,1,2018,4.000000,7,0.0,8.666667,65.333333,64.000000,...,800.333333,1022.333333,800.000000,1013.148351,287.150000,1032.333333,801.333333,41.000000,286.150000,286.816667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,9,31,12,2018,0.333333,9,0.0,2.000000,81.666667,49.666667,...,800.000000,1034.666667,800.000000,1013.148351,280.816667,1031.666667,734.333333,69.666667,276.483333,276.150000
2916,12,31,12,2018,0.333333,8,0.0,1.333333,61.000000,28.333333,...,800.000000,1034.333333,800.000000,1013.148351,287.483333,1031.333333,747.666667,32.000000,285.816667,278.816667
2917,15,31,12,2018,1.000000,6,0.0,3.000000,47.000000,26.333333,...,800.000000,1033.000000,800.000000,1013.148351,290.816667,1030.000000,800.000000,23.666667,288.150000,285.816667
2918,18,31,12,2018,1.000000,6,0.0,2.000000,52.666667,56.666667,...,800.000000,1033.333333,800.000000,1013.148351,288.816667,1030.333333,800.000000,59.000000,283.150000,282.816667


<br>

### Loading the trained Model

In [13]:
# Load the model from the file

model_filename = 'random_forest_model.pkl'

with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

### Making the Prediction using the loaded model

In [14]:
submitted_predictions = loaded_model.predict(test_data)
submitted_predictions

array([11302.        ,  9823.24666667,  9433.83666667, ...,
       13283.91139702, 14494.94299801, 15028.33037037])

In [15]:
submitted_predictions_df = pd.DataFrame(submitted_predictions, columns=['Predicted_load_shortfall_3h'])
submitted_predictions_df

,Predicted_load_shortfall_3h
0,11302.000000
1,9823.246667
2,9433.836667
3,9653.133333
4,13206.648333
...,...
2915,12542.595833
2916,14486.052128
2917,13283.911397
2918,14494.942998


In [16]:
submitted_predictions_df.describe().round()

,Predicted_load_shortfall_3h
count,2920.0
mean,11537.0
std,3265.0
min,1200.0
25%,9040.0
50%,11842.0
75%,13979.0
max,20110.0


### Saving the Model as a csv file

In [17]:
output = pd.DataFrame({'time':df_test['time']})
team_submission = output.join(submitted_predictions_df)
team_submission.to_csv('team_NM2_submission_new.csv', index = False)